In [1]:
import pandas as pd
import numpy as np
new_lab = pd.read_csv("df_labdata.csv")
new_lab.rename(columns={"test_result": "result", "test_unit": 'Units'}, inplace = 'True')
old_lab = pd.read_csv('TMA_lab.csv')
old_lab['Lab_Date'] = pd.to_datetime(old_lab.Lab_Date)
old_lab.rename(columns={"Lab_Date": "date", "Results": "result", "Code": "test_name"}, inplace = 'True')
master = pd.read_csv("master.csv")
masnew = pd.read_csv('master_new.csv')
masnew.rename(columns={"ACCT": "TMA_Acct"}, inplace = 'True')
stage_base = pd.read_csv('stage_base.csv')
def num(df):
    print(len(df))
    print(len(df['TMA_Acct'].unique()))
def show(df, n):
    print(pd.DataFrame(df[df['TMA_Acct'] == n]))
pth_name = ['intact pth', 'ipth', 'calc (pth)', 'parathyrin.intact', 'parathyroid hormone, intact', 'pth', 'pth calcium', 'pth in', 'pth intact', 'pth, intact', 'pth,int', 'pth,intact'] 

In [4]:
new_pth = new_lab[new_lab['test_name'].isin(pth_name)][['TMA_Acct','date', 'test_name', 'result', 'Units']]
old_pth = old_lab[old_lab['test_name'].isin(pth_name)][['TMA_Acct', 'date', 'test_name', 'result', 'Units']]

In [5]:
pth = pd.concat([old_pth, new_pth], axis = 0)
pth = pth.sort_values(['TMA_Acct', 'date'])

In [6]:
pth['date'] = pd.to_datetime(pth['date'])
pth = pth[(pth['date'] > '2009-12-31') & (pth['date'] < '2019-01-01')]
pth = pth.drop_duplicates()
pth = pth[pth['Units'] != 'HPF']
pth = pth[pth['Units'] != 'LPF']

In [7]:
len(pth)

24778

In [8]:
delete = ['TNP', 'PTH-RP12','None', 'Normal','<4.0','Pending Results','not done','Negative','*','No Specimen Rec','No Result', 'See Scanned', '(See Note)', 'Not Drawn', 'Normal See Scan','Not Performed', 'Canceled', 'Not Done', 'Comment', 'Ignore Results,', 'NR', 'Pending', 'See Note']
pth = pth[~pth['result'].isin(delete)]

In [9]:
pth['result'] = pth['result'].replace(['172.0 *'],'172.0')
pth['result'] = pth['result'].replace(['68..0'],'68.0')
pth['result'] = pth['result'].astype('float')
pth = pth.dropna(subset = ['result'])

In [10]:
print(len(pth))
pth = pth[pth['result'] < 300]
pth = pth[pth['result'] > 35]
print(len(pth))

21977
17957


In [9]:
def final_data(data):
    early = pd.DataFrame(data.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
    early.columns=['TMA_Acct', 'min']
    data = pd.merge(data, early, how = 'left', left_on = ['TMA_Acct'], right_on = ['TMA_Acct'])
    data['date'] = pd.to_datetime(data['date'])
    data['min'] = pd.to_datetime(data['min'])
    data['period'] = ((data['date'] - data['min']).dt.days / 90).astype('int')
    data_traj = data[['TMA_Acct', 'result', 'period']]
    data_traj = pd.DataFrame(data_traj.groupby(['TMA_Acct', 'period']).agg({'result': np.mean})).reset_index()
    data_traj = data_traj.pivot(index='TMA_Acct', columns='period', values='result')
    print(len(data_traj))
    final_data = data_traj.dropna(thresh=3)
    print(len(final_data))
    for i in range(37):
        final_data['t' + str(i)] = i
        final_data.rename(columns={i: "v" + str(i)}, inplace = 'True')
    return final_data

In [10]:
final_pth = final_data(pth)

4769
2090


C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
final_pth

period,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36
TMA_Acct,,,,,,,,,,,,,,,,,,,,,
37,93.0,187.0,183.0,191.0,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
86,48.2,105.0,112.0,NaN,60.7,NaN,84.2,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
97,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
154,50.0,NaN,45.0,NaN,NaN,39.0,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
214,98.0,NaN,NaN,91.2,NaN,NaN,NaN,NaN,103.6,NaN,...,27,28,29,30,31,32,33,34,35,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49911,44.0,NaN,NaN,57.0,NaN,65.0,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
50179,69.0,NaN,81.0,108.0,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
50284,107.0,NaN,NaN,NaN,109.0,64.6,NaN,NaN,243.0,96.5,...,27,28,29,30,31,32,33,34,35,36


In [12]:
def stage_data(stage_base, data, name):
    stage_data = pd.merge(data, stage_base, left_on = 'TMA_Acct', right_on = 'TMA_Acct')
    stage_data_3a = stage_data[stage_data['stage'] == '3a']
    stage_data_3b = stage_data[stage_data['stage'] == '3b']
    stage_data_4 = stage_data[stage_data['stage'] == '4']
    stage_data_3a.to_csv('stage_'+name +'_3a.csv')
    print(len(stage_data_3a))
    stage_data_3b.to_csv('stage' + name +'_3b.csv')
    print(len(stage_data_3b))
    stage_data_4.to_csv('stage'+name+'_4.csv')
    print(len(stage_data_4))

In [13]:
stage_data(stage_base, final_pth, 'pth')

330
514
332


In [2]:
stage1 = pd.read_csv("stage_pth_3a.csv")

In [3]:
stage1

,Unnamed: 0,TMA_Acct,v0,v1,v2,v3,v4,v5,v6,v7,...,t34,t35,t36,Unnamed: 0.1,Age_in_2010,sex_v,race_v,hyper_v,diabe_v,stage
0,2,97,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34,35,36,0,76,1,0,1,1,3a
1,29,1375,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34,35,36,8,80,0,0,1,0,3a
2,30,1407,73.0,NaN,71.0,NaN,76.0,78.5,NaN,NaN,...,34,35,36,9,67,1,0,1,1,3a
3,31,1412,84.6,NaN,NaN,84.2,NaN,NaN,NaN,NaN,...,34,35,36,10,77,0,0,1,0,3a
4,36,1630,39.5,53.8,NaN,NaN,NaN,81.6,NaN,77.9,...,34,35,36,12,74,1,0,1,0,3a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,1147,47248,100.5,NaN,141.9,105.0,NaN,NaN,NaN,NaN,...,34,35,36,840,67,0,0,0,0,3a
326,1154,47837,59.0,115.0,73.0,NaN,NaN,NaN,NaN,NaN,...,34,35,36,847,42,0,0,1,0,3a
327,1155,47852,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34,35,36,848,63,0,0,0,0,3a
328,1159,47981,52.0,NaN,51.0,NaN,61.0,NaN,NaN,NaN,...,34,35,36,851,66,0,0,0,0,3a


In [17]:
def sumarry_data(stage_base, data):
    stage_data = pd.merge(data, stage_base, left_on = 'TMA_Acct', right_on = 'TMA_Acct')
    stage_data_3a = stage_data[stage_data['stage'] == '3a']
    print(stage_data_3a['result'].describe())
    stage_data_3b = stage_data[stage_data['stage'] == '3b']
    print(stage_data_3b['result'].describe())
    stage_data_4 = stage_data[stage_data['stage'] == '4']
    print(stage_data_4['result'].describe())
sumarry_data(stage_base, pth)

count    2879.000000
mean       79.494168
std        41.599099
min        35.100000
25%        51.000000
50%        68.000000
75%        93.950000
max       299.000000
Name: result, dtype: float64
count    4537.000000
mean       89.805356
std        49.240834
min        35.100000
25%        54.400000
50%        74.900000
75%       109.000000
max       299.400000
Name: result, dtype: float64
count    3229.000000
mean      101.668194
std        52.206791
min        35.100000
25%        63.000000
50%        89.000000
75%       127.000000
max       298.400000
Name: result, dtype: float64


In [12]:
df = pd.merge(pth, stage_base, left_on = 'TMA_Acct', right_on = 'TMA_Acct')
df.describe()

,TMA_Acct,result,Unnamed: 0,Age_in_2010,sex_v,race_v,hyper_v,diabe_v
count,10645.000000,10645.000000,10645.000000,10645.000000,10645.000000,10645.000000,10645.000000,10645.000000
mean,24050.458055,90.615050,348.052983,71.360545,0.472616,0.030531,0.932363,0.425646
std,13505.011570,48.975017,243.466160,11.323247,0.499273,0.172051,0.251134,0.494464
min,37.000000,35.100000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000
25%,11891.000000,55.000000,153.000000,66.000000,0.000000,0.000000,1.000000,0.000000
50%,26656.000000,76.000000,297.000000,73.000000,0.000000,0.000000,1.000000,0.000000
75%,34893.000000,111.000000,503.000000,79.000000,1.000000,0.000000,1.000000,1.000000
max,50855.000000,299.400000,1019.000000,98.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df["result"].mean()